In [1]:
# Cell 0: Setup
import os
import random
import cv2  # OpenCV for video processing
import time

# For inference with YOLO (assuming you use ultralytics)
from ultralytics import YOLO

# Define directory paths (modify as per your folder structure)
test_videos_dir = 'data/test'  # Directory containing your test videos
extracted_frames_dir = 'data/test_frames'  # Directory to save extracted frames

# Create folder if it doesn't exist
os.makedirs(extracted_frames_dir, exist_ok=True)


In [71]:
# select 3 random vids
import random
random.seed(162)

# Cell 1: Randomly choose 3 candidate videos
all_videos = [f for f in os.listdir(test_videos_dir) if f.lower().endswith(('.mp4', '.avi', '.mov'))]
print(f"Total videos found: {len(all_videos)}")

# Select 3 random videos (ensure there are at least 3)
num_candidates = 3
candidate_videos = random.sample(all_videos, min(num_candidates, len(all_videos)))
print("Selected candidate videos:")
for vid in candidate_videos:
    print(vid)


Total videos found: 10
Selected candidate videos:
bicycle_test.mp4
car_test.mp4
people_test.mp4


In [72]:
# Cell 2: Extract one frame from each segment
# Define the segment length and the gap between segments
segment_length = 6   # seconds per segment
segment_gap = 1      # gap between segments, so segments start at 0, 7, 13, etc.

# We'll choose the middle of each segment. For a 6-sec segment, the midpoint is 3 seconds after the segment's start.
def extract_segment_frames(video_path, output_dir, seg_length, seg_gap):
    cap = cv2.VideoCapture(video_path)
    
    # Get FPS and total frame count
    video_fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    duration_sec = total_frames / video_fps
    print(f"Video: {os.path.basename(video_path)} | FPS: {video_fps:.2f} | Duration: {duration_sec:.2f}s")
    
    # Calculate the start times for segments; segments start at times 0, (seg_length + seg_gap), (2*(seg_length+seg_gap)), etc.
    segment_interval = seg_length + seg_gap
    segment_starts = [t for t in range(0, int(duration_sec), segment_interval) if t + seg_length <= duration_sec]
    
    print(f"Number of segments to extract: {len(segment_starts)}")
    
    # For each segment, choose the midpoint frame (start time + seg_length/2)
    for seg_start in segment_starts:
        target_time = seg_start + seg_length / 2  # in seconds
        target_frame_index = int(target_time * video_fps)
        
        # Set the video capture position to the target frame index
        cap.set(cv2.CAP_PROP_POS_FRAMES, target_frame_index)
        ret, frame = cap.read()
        if ret:
            # Define filename and save the extracted frame
            frame_filename = os.path.join(output_dir, f"frame_{target_frame_index:04d}.jpg")
            cv2.imwrite(frame_filename, frame)
            print(f"Saved frame at {target_time:.1f}s (Frame {target_frame_index}) -> {frame_filename}")
        else:
            print(f"Failed to capture frame at {target_time:.1f}s (Frame {target_frame_index})")
    
    cap.release()

# Process each candidate video
for vid_name in candidate_videos:
    vid_path = os.path.join(test_videos_dir, vid_name)
    # Create a subdirectory for frames for this video
    video_frames_dir = os.path.join(extracted_frames_dir, os.path.splitext(vid_name)[0])
    os.makedirs(video_frames_dir, exist_ok=True)
    
    # Extract frames with non-continuous segments
    extract_segment_frames(vid_path, video_frames_dir, segment_length, segment_gap)
    print()


Video: bicycle_test.mp4 | FPS: 28.00 | Duration: 79.11s
Number of segments to extract: 11
Saved frame at 3.0s (Frame 84) -> data/test_frames\bicycle_test\frame_0084.jpg
Saved frame at 10.0s (Frame 280) -> data/test_frames\bicycle_test\frame_0280.jpg
Saved frame at 17.0s (Frame 476) -> data/test_frames\bicycle_test\frame_0476.jpg
Saved frame at 24.0s (Frame 672) -> data/test_frames\bicycle_test\frame_0672.jpg
Saved frame at 31.0s (Frame 868) -> data/test_frames\bicycle_test\frame_0868.jpg
Saved frame at 38.0s (Frame 1064) -> data/test_frames\bicycle_test\frame_1064.jpg
Saved frame at 45.0s (Frame 1260) -> data/test_frames\bicycle_test\frame_1260.jpg
Saved frame at 52.0s (Frame 1456) -> data/test_frames\bicycle_test\frame_1456.jpg
Saved frame at 59.0s (Frame 1652) -> data/test_frames\bicycle_test\frame_1652.jpg
Saved frame at 66.0s (Frame 1848) -> data/test_frames\bicycle_test\frame_1848.jpg
Saved frame at 73.0s (Frame 2044) -> data/test_frames\bicycle_test\frame_2044.jpg

Video: car_tes

# TEST CLASS CONVERTER

In [ ]:
# TEST
# names: ['Bicycle', 'Car', 'Motorbike', 'People']

# ACTUAL

# Inference


In [ ]:
from sklearn.metrics import average_precision_score

# Cell 4: Run inference on all images in a folder, measure inference time, and calculate mAP


# Load your pre-trained YOLO model (update the model path accordingly)
model = YOLO("C:\\Users\\redoks\\Documents\\skripzii\\products6k\\YOLOv9\\YOLOv9s\\batch8_lr0.01\\weights\\best.pt")  # Replace with your model file

def run_inference_on_image(image_path):
    img = cv2.imread(image_path)
    start_time = time.time()
    # Run prediction; here we use conf=0.25 as default
    results = model.predict(source=img, conf=0.489, imgsz=640)
    elapsed = time.time() - start_time
    return results, elapsed

def parse_results(results):
    """Extract bounding boxes and classes from YOLO results."""
    boxes = results[0].boxes.xyxy.cpu().numpy()  # Bounding boxes (x1, y1, x2, y2)
    scores = results[0].boxes.conf.cpu().numpy()  # Confidence scores
    classes = results[0].boxes.cls.cpu().numpy()  # Class IDs
    return boxes, scores, classes

def parse_ground_truth(label_path):
    """Parse ground truth labels from a YOLO-format label file."""
    with open(label_path, 'r') as f:
        lines = f.readlines()
    gt_boxes = []
    gt_classes = []
    for line in lines:
        parts = line.strip().split()
        cls = int(parts[0])  # Class ID
        x_center, y_center, width, height = map(float, parts[1:])
        gt_classes.append(cls)
        # Convert YOLO format (x_center, y_center, width, height) to (x1, y1, x2, y2)
        x1 = x_center - width / 2
        y1 = y_center - height / 2
        x2 = x_center + width / 2
        y2 = y_center + height / 2
        gt_boxes.append([x1, y1, x2, y2])
    return gt_boxes, gt_classes

# Define the folder containing images and labels
images_dir = 'data/test_ready/test/images'
labels_dir = 'data/test_ready/test/labels'

# Get all image files in the folder
image_files = sorted([f for f in os.listdir(images_dir) if f.endswith('.jpg') or f.endswith('.png')])
print(f"Found {len(image_files)} images in {images_dir}")

# Run inference on all images
all_inference_times = []
all_aps = []  # Store average precision for each image
for image_file in image_files:
    image_path = os.path.join(images_dir, image_file)
    label_path = os.path.join(labels_dir, os.path.splitext(image_file)[0] + '.txt')  # Corresponding label file

    # Check if the label file exists
    if not os.path.exists(label_path):
        print(f"Warning: Label file not found for {image_file}. Skipping...")
        continue

    # Run inference
    results, elapsed = run_inference_on_image(image_path)
    all_inference_times.append(elapsed)

    # Parse results and ground truth
    pred_boxes, pred_scores, pred_classes = parse_results(results)
    gt_boxes, gt_classes = parse_ground_truth(label_path)

    # Calculate mAP (for simplicity, we calculate AP per class and average them)
    aps = []
    for cls in set(gt_classes + list(pred_classes)):
        gt_binary = [1 if c == cls else 0 for c in gt_classes]
        # Filter pred_scores for predictions matching the current class:
        pred_scores_cls = [score for score, c in zip(pred_scores, pred_classes) if c == cls]
        # Also create a pred_binary that has the same length as pred_scores_cls (typically all ones, since these are only predictions for this class)
        pred_binary = [1] * len(pred_scores_cls)

        if sum(gt_binary) > 0 and len(pred_scores_cls) > 0:
            aps.append(average_precision_score(gt_binary, pred_scores_cls))


    # Print results
    print(f"Image {image_file}: Inference time: {elapsed*1000:.2f} ms")
    print(f"Results: {results}")  # You can process results further if needed

# Calculate and display average inference time and mAP
if all_inference_times:
    avg_time = sum(all_inference_times) / len(all_inference_times)
    print(f"Average inference time per image: {avg_time*1000:.2f} ms")

if all_aps:
    mean_ap = sum(all_aps) / len(all_aps)
    print(f"Mean Average Precision (mAP): {mean_ap:.4f}")


# 2nd way to inference

In [6]:
from ultralytics import YOLO
import time

# === Load your trained model ===
model = YOLO("C:\\Users\\redoks\\Documents\\skripzii\\products6k\\YOLOv9\\YOLOv9s\\batch8_lr0.01\\weights\\best.pt")  # Replace with your model file

# === Start timer for full validation phase ===
start_time = time.time()

# === Run validation using the test set ===
results = model.val(
    data=r"C:\Users\redoks\Documents\skripzii\data\test_ready\data.yaml",
    split='test',       # Force it to use the test split
    conf=0.489,           # Confidence threshold
    iou=0.5,             # IoU threshold
    imgsz=640,           # Image size (default from YOLO config)
    max_det=300,         # Max detections per image
    device='cpu'           # Set to 0 for GPU, or 'cpu' if needed
)

# === End timer ===
end_time = time.time()

# === Print relevant metrics ===
print("\n=== Evaluation Results ===")
print(f"mAP@0.5:        {results.box.map50:.4f}")
print(f"mAP@0.5:0.95:   {results.box.map:.4f}")
print(f"Inference Time: {results.speed['inference']:.2f} ms/image")
print(f"Total Time:     {end_time - start_time:.2f} seconds")


Ultralytics 8.3.107  Python-3.12.0 torch-2.6.0+cpu CPU (Intel Core(TM) i5-8265U 1.60GHz)
YOLOv9s summary (fused): 197 layers, 7,171,732 parameters, 0 gradients, 26.8 GFLOPs


FileNotFoundError: 
Dataset 'C://Users/redoks/Documents/skripzii/data/test_ready/data.yaml' images not found ⚠️, missing path 'C:\Users\redoks\Documents\skripzii\data\test_ready\valid\images'
Note dataset download directory is 'C:\Users\redoks\Documents\skripzii\datasets'. You can update this in 'C:\Users\redoks\AppData\Roaming\Ultralytics\settings.json'

In [ ]:
# Cell 5: (Optional) Evaluate mAP using the YOLO test command (run in a terminal or via subprocess)
!yolo task=detect mode=test model=path/to/your/trained_model.pt data=path/to/dataset.yaml imgsz=640 conf=0.25
